Get Scenes from https://drive.google.com/drive/folders/1URQoHuySdfWeURLd0LiXsuWwUtfMJWA1


In [ ]:
import trimesh
import pyrender
import numpy as np
from PIL import Image
import os

import pyrender.constants

# os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [7]:
SLUG_MESH = trimesh.scene.Scene(trimesh.load('slug_mesh.glb'))

In [17]:
def look_at(eye, target, up):
    forward = target - eye
    forward = forward / np.linalg.norm(forward)
    right = np.cross(up, forward)
    right = right / np.linalg.norm(right)
    up = np.cross(forward, right)
    up = up / np.linalg.norm(up)
    
    matrix = np.eye(4)
    matrix[0, :3] = right
    matrix[1, :3] = up
    matrix[2, :3] = forward
    matrix[:3, 3] = -np.dot(np.array([right, up, forward]), eye)
    
    return matrix

def create_slug_render(img_size=(512, 512)):
    scene = pyrender.Scene.from_trimesh_scene(SLUG_MESH)
    scene.bg_color = [0, 0, 0, 0]  

    cam_pos = np.random.uniform(-2, 2, size=3)
    cam_target = np.zeros(3)
    up = np.array([0, 1, 0])
    
    camera_pose = look_at(cam_pos, cam_target, up)
    
    camera = pyrender.PerspectiveCamera(yfov=np.random.uniform(0.5, 1.5))
    scene.add(camera, pose=camera_pose)

    renderer = pyrender.OffscreenRenderer(*img_size)
    color, _ = renderer.render(scene, flags = pyrender.constants.RenderFlags.RGBA)
    renderer.delete()

    print(color.shape)
    img = Image.fromarray(color, 'RGBA')
    return img

def create_datapoint(foreground, background_path):
    background = Image.open(background_path).convert("RGBA")

    scale = min(np.random.uniform(0.3, 1.0), 
                min(background.width / foreground.width, background.height / foreground.height) * 0.9)  # Added * 0.9 for a small margin
    new_size = (int(foreground.width * scale), int(foreground.height * scale))
    foreground = foreground.resize(new_size)

    max_x = background.width - foreground.width
    max_y = background.height - foreground.height
    pos_x = np.random.randint(0, max_x)
    pos_y = np.random.randint(0, max_y)

    label = [
        pos_x,                    
        pos_y,                    
        pos_x + foreground.width, 
        pos_y + foreground.height 
    ]

    background.paste(foreground, (pos_x, pos_y), foreground)
    return background, label



In [9]:
SYNTHETIC_DATA_DIR = "synthetic-data"

os.makedirs(SYNTHETIC_DATA_DIR, exist_ok=True)

BACKGROUND_IMAGE_DIR = "backgrounds"
image_names = os.listdir(BACKGROUND_IMAGE_DIR)
image_files = [os.path.join(BACKGROUND_IMAGE_DIR, name) for name in image_names if name.endswith(".jpg")]

NUM_DATAPOINTS = 10

In [ ]:
YAML_FILE = """
names:
- slug
nc: 1
test: ../test/images
train: ../train/images
val: ../valid/images
"""

with open(os.path.join(SYNTHETIC_DATA_DIR, 'data.yaml'), "w") as f:
    f.write(YAML_FILE)

TRAIN_SPLIT = 0.7
TEST_SPLIT = 0.2

train_index = int(NUM_DATAPOINTS * TRAIN_SPLIT)
test_index = int(NUM_DATAPOINTS * (1 - TEST_SPLIT))

indices = [train_index, test_index, NUM_DATAPOINTS - 1]

index = 0
for i, split in enumerate(["train", "valid", "test"]):
    os.makedirs(os.path.join(SYNTHETIC_DATA_DIR, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(SYNTHETIC_DATA_DIR, split, "labels"), exist_ok=True)
    final_index = indices[i]

    while index < final_index:
        rendered_model = create_slug_render()
        final_image, bbox = create_datapoint(rendered_model, image_files[index])
        final_image.show()  
        final_image.save(os.path.join(SYNTHETIC_DATA_DIR, split, 'images', "{}.png".format(index)))

        with open(os.path.join(SYNTHETIC_DATA_DIR, split, "labels", "{}.txt".format(index)), "w") as f:
            f.write("0 {} {} {} {}".format(*bbox))

        del rendered_model

        index += 1